In [1]:
from load_data import *
import L0_card
import numpy as np
import copy
import matplotlib.pyplot as plt

In [2]:
sample_size = 1000
batch_size = 16
seed = 0
model_name = "mnist"

In [3]:
X, w_bar, model, train_dataloader, test_dataloader, modules_to_prune = load_Xw(model_name, sample_size, batch_size, seed)

Started logging run 20220705_13-35-01_426047_64 of experiment mlpnet, saving checkpoints every 1 epoch
this is config before OrderedDict([('pruners', OrderedDict([('pruner_1', OrderedDict([('class', 'WoodburryFisherPruner'), ('epochs', [0, 2, 2]), ('weight_only', True), ('initial_sparsity', 0.0), ('target_sparsity', 0.9), ('modules', ['fc2']), ('keep_pruned', True)]))])), ('trainers', OrderedDict([('default_trainer', OrderedDict([('optimizer', OrderedDict([('class', 'SGD'), ('lr', 0.001), ('momentum', 0.5)])), ('lr_scheduler', OrderedDict([('class', 'ExponentialLR'), ('gamma', 0.9), ('epochs', [120, 1, 140])]))]))]))])
Overwrite arguments for one-shot!
this is config after OrderedDict([('pruners', OrderedDict([('pruner_1', OrderedDict([('class', 'BlockwiseWoodburryFisherPruner'), ('epochs', [0, 1, 1]), ('weight_only', True), ('initial_sparsity', 0.9), ('target_sparsity', 0.9), ('modules', ['fc1', 'fc2', 'fc3']), ('keep_pruned', True)]))])), ('trainers', OrderedDict([('default_trainer',

Also printing results@top5 as well!
Do log softmax:  True
=======Layers:=============


/home/mengx/.local/lib/python3.8/site-packages/torch/cuda/__init__.py:122: UserWarning: 
    Found GPU0 Tesla K20m which is of cuda capability 3.5.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability supported by this library is 3.7.
    
  warnings.warn(old_gpu_warn % (d, name, major, minor, min_arch // 10, min_arch % 10))
Module .fc1 was successfully wrapped
Module .fc2 was successfully wrapped
Module .fc3 was successfully wrapped


current_model_config {'arch': 'mlpnet', 'dataset': 'mnist', 'use_butterfly': False}
Do log softmax:  True
updated_state_dict odict_keys(['fc1._weight_mask', 'fc1._layer.weight', 'fc2._weight_mask', 'fc2._layer.weight', 'fc3._weight_mask', 'fc3._layer.weight'])
args is  Namespace(aa=False, always_eval_test=False, arch='mlpnet', aux_gpu_id=-1, batch_size=64, batched_test=False, cache_subset_ids=False, centered=False, check_grads=False, check_train_loss=False, checkpoint_freq=1, ckpt_epoch=-1, compare_globalmagni_mask=False, compare_models=False, config_path='../WoodFisher/configs/mlpnet_mnist_config_one_shot_woodburry_fisher.yaml', cpu=True, deterministic=True, device=device(type='cpu'), disable_bias=True, disable_log_soft=False, disable_train_random_transforms=False, disable_train_shuffle=False, disable_wdecay_after_prune=False, dset='mnist', dset_path='./datasets', dump_fisher_inv_mat=False, dump_grads_mat=False, enable_dropout=False, epochs=1, eps=1e-10, eval_fast=False, exp_dir='../e

Constructed BlockwiseWoodburryFisherPruner with config:
    -epochs:[0, 1, 1]
    -weight_only:True
    -initial_sparsity:0.9
    -target_sparsity:0.9
    -modules:['fc1', 'fc2', 'fc3']
    -keep_pruned:True

IN BLOCK WOODBURRY
initial sparsity in args is 0.9
initial sparsity in config is 0.9
../WoodFisher/models/mlpnet.py:45: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
  return F.log_softmax(x)


---computing gradients--  0
---computing gradients--  100
---computing gradients--  200
---computing gradients--  300
---computing gradients--  400
---computing gradients--  500
---computing gradients--  600
---computing gradients--  700
---computing gradients--  800
---computing gradients--  900


In [4]:
y = X@w_bar
p = w_bar.shape[0]
beta = w_bar
r = y - X@beta
sparsity = 0.9
ntot = w_bar.shape[0]
ntar = int(ntot * (1-sparsity))
print("ntar is ",ntar)
print("Shape of X is ", X.shape)

ntar is  3235
Shape of X is  (1000, 32360)


Important Parameters:

+ k: number of non-zeros
+ alpha,lambda1,lambda2,M, beta_tilde1,beta_tilde2: parameters of the objective $F(\beta)$ (see Cardinality-heuristics.tex in overleaf)
+ L: estimation of Lipschitz condition of $\nabla f(\beta)$ (just set L=None and the alg will compute it by itself)
+ iht_max_itr: maximal number of inner iterations
+ ftol: stopping criteria of inner loop
+ act_max_itr: maximal number of outer iterations
+ buget,kimp: the initial cardinality of the active set is set to be $\max(k, \min(p,kimp*k,buget))$
+ act_itr: initial IHT iterations used to generate the active set
+ cd_itr: number of CD iteration performed in each inner loop
+ ctol: starting criteria of CD iteration 
+ sea1_max_itr: number of line search performed in each inner loop
+ sea2_max_itr: number of line search performed when updating the active set

In [14]:
beta = np.copy(w_bar)
k = ntar
beta, f, objs, r, cur_iter, sol_time = L0_card.Vanilla_IHTCD(y,X,beta,k,alpha=np.zeros(p),lambda1=0.001,lambda2=0.01,M=np.inf, beta_tilde1=np.copy(w_bar),
            beta_tilde2=np.copy(w_bar), L=None, iht_max_itr=600,ftol=1e-6,cd_itr=1,ctol=1e-4,search_max_itr=5)

In [15]:
sol_time, f , cur_iter

(186.94732856750488, 0.7746456911085802, 600)

In [16]:
check_imp(model, beta, test_dataloader, modules_to_prune=modules_to_prune)

(tensor(0.), 93.97, tensor(0.9000), 92.94)

In [5]:
beta = np.copy(w_bar)
k = ntar
beta, f, sols, r, cur_iter, sol_time = L0_card.Active_IHTCD(y,X,beta,k,alpha=np.zeros(p),lambda1=0.001,lambda2=0.01,M=np.inf, beta_tilde1=np.copy(w_bar), 
                     beta_tilde2=np.copy(w_bar), L=None, iht_max_itr=600, ftol = 1e-6, act_max_itr=10, buget=None, kimp=2, act_itr=100,
                     cd_itr = 5, ctol = 1e-4, sea1_max_itr=5, sea2_max_itr=10)

/home/mengx/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


Num of iter: 1  num of inner iter: 600 
 Finding new active set
0.7749946368206647 0.7749946640805259 0 1
0.7749946126842256 0.7749946640805259 0 2
0.7749945737816218 0.7749946640805259 0 3
0.7749945334574897 0.7749946640805259 0 4
0.7749946027335374 0.7749946640805259 0 5
0.7749953409828836 0.7749946640805259 0 6


In [6]:
sol_time, f , cur_iter

(29.032485723495483, 0.7749946640805259, 0)

In [7]:
check_imp(model, beta, test_dataloader, modules_to_prune=modules_to_prune)

(tensor(0.), 93.97, tensor(0.9000), 93.06)

In [ ]:
# how to deal the case in which X has zero column 

In [3]:
# the first 100 columen of X is 1
np.random.seed(seed=2)
X = np.hstack([np.zeros((100,100)),np.random.randn(100,1000)])
w_bar = np.random.randn(1100)
y = X@w_bar
p = 1100

In [3]:
# direct apply IHTCD (or active IHTCD) yields divide by 0 error

beta = np.copy(w_bar)
beta, f, sols, r, cur_iter, sol_time = L0_card.Active_IHTCD(y,X,beta,100,alpha=np.zeros(p),lambda1=0.001,lambda2=0,M=np.inf, beta_tilde1=np.copy(w_bar), 
                     beta_tilde2=np.copy(w_bar), L=None, iht_max_itr=600, ftol = 1e-6, act_max_itr=10, buget=None, kimp=2, act_itr=100,
                     cd_itr = 5, ctol = 1e-4, sea1_max_itr=5, sea2_max_itr=10)

/home/mengx/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(
/home/mengx/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


ZeroDivisionError: division by zero

In [4]:
# Apply IHTCD (or active IHTCD) with a pre-processing procedure solves the problem

beta = np.copy(w_bar)
beta, f, sols, r, cur_iter, sol_time = L0_card.Active_IHTCD_PP(y,X,beta,100,alpha=np.zeros(p),lambda1=0.001,lambda2=0,M=np.inf, beta_tilde1=np.copy(w_bar), 
                     beta_tilde2=np.copy(w_bar), L=None, iht_max_itr=600, ftol = 1e-6, act_max_itr=10, buget=None, kimp=2, act_itr=100,
                     cd_itr = 5, ctol = 1e-4, sea1_max_itr=5, sea2_max_itr=10)

/home/mengx/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(
/home/mengx/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(


Num of iter: 1  num of inner iter: 600 
 Finding new active set
2.0310441597160707 2.0335625093232723 0 1
2.0287579987827176 2.0335625093232723 0 2
2.0248822429374096 2.0335625093232723 0 3
2.0199169953325575 2.0335625093232723 0 4
2.0211315564658143 2.0335625093232723 0 5
2.068140904104333 2.0335625093232723 0 6


In [8]:
beta, f, objs, r, iht_cur_itr, sol_time = L0_card.Vanilla_IHTCD_PP(y,X,beta,100,alpha=np.zeros(p),lambda1=0.001,lambda2=0,M=np.inf, beta_tilde1=np.copy(w_bar), 
                     beta_tilde2=np.copy(w_bar), L=None, iht_max_itr=600, ftol = 1e-6, cd_itr=0,ctol=1e-4,search_max_itr=1)

/home/mengx/.local/lib/python3.8/site-packages/sklearn/utils/extmath.py:368: FutureWarning: If 'random_state' is not supplied, the current default is to use 0 as a fixed seed. This will change to  None in version 1.2 leading to non-deterministic results that better reflect nature of the randomized_svd solver. If you want to silence this warning, set 'random_state' to an integer seed or to None explicitly depending if you want your code to be deterministic or not.
  warnings.warn(
